# Clustering Analysis on Merchant Cost Patterns

## Notebook Overview

This notebook performs multiple clustering methodologies on merchant-level features derived from transaction cost data for MCC 5411 (Grocery Stores).

### Features Used
- `cost_percent`: Mean processing cost as percentage of transaction amount per merchant
- `cost_percent_stdev`: Standard deviation of cost percentage per merchant

### Datasets
- **Raw Data**: `df_*_base.csv` - Original values without normalization
- **Z-Scored Data**: `df_*_base_zscored.csv` - Pre-normalized values
- **Splits**: Train (2017), Validate (2018), Test (2019)

### Clustering Methods
1. **K-Means** - Partition-based clustering with k=2-15 testing
2. **Hierarchical Clustering** - Agglomerative clustering with dendrogram visualization
3. **DBSCAN** - Density-based clustering with noise detection

### Analysis Steps
1. **Data Loading & Quality Checks** - Load data, handle NaN/infinity values
2. **K-Means Clustering** - Test different k values, compare raw vs z-scored
3. **Hierarchical Clustering** - Ward linkage, dendrogram analysis
4. **DBSCAN** - Density-based clustering, epsilon tuning
5. **Methods Comparison** - Compare all clustering approaches

### Evaluation Metrics
- **Silhouette Score**: Measures cluster cohesion and separation (higher is better, range [-1, 1])
- **Davies-Bouldin Index**: Measures cluster similarity (lower is better, range [0, ∞))
- **Inertia**: Within-cluster sum of squares (elbow method)

---


## 1. Setup & Imports


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## 2. Data Loading & Quality Checks (Raw Data)


In [ ]:
# load datasets

df_train = pd.read_csv('base_features/df_train_base.csv')
df_validate = pd.read_csv('base_features/df_validate_base.csv')
df_test = pd.read_csv('base_features/df_test_base.csv')

In [ ]:
df_validate.describe()

In [ ]:
# Diagnostic: Check for infinity/NaN in raw loaded data
print("=== CHECKING RAW DATA FROM CSV ===\n")

for name, df in [('Train', df_train), ('Validate', df_validate), ('Test', df_test)]:
    print(f"\n{name} dataset:")
    print(f"  Shape: {df.shape}")
    print(f"  NaN count: {df.isna().sum().sum()}")
    print(f"  Inf count: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")
    
    # Check actual min/max values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        col_min = df[col].min()
        col_max = df[col].max()
        if np.isinf(col_min) or np.isinf(col_max) or abs(col_max) > 1e10:
            print(f"  WARNING - {col}: min={col_min}, max={col_max}")
            # Show which merchants have infinity
            inf_rows = np.isinf(df[col])
            if inf_rows.sum() > 0:
                print(f"    Merchants with infinity in {col}: {df.loc[inf_rows, 'merchant_id'].tolist()[:10]}")

In [ ]:
# Investigate merchant 50783 specifically
print("=== INVESTIGATING MERCHANT 50783 ===\n")

merchant_id = 50783

# Check in each dataset
for name, df in [('Train', df_train), ('Validate', df_validate), ('Test', df_test)]:
    if merchant_id in df['merchant_id'].values:
        print(f"\nFound in {name} dataset:")
        merchant_data = df[df['merchant_id'] == merchant_id]
        print(merchant_data.T)  # Transpose for easier reading
        
        # Check for inf/nan
        print(f"\nHas NaN: {merchant_data.isna().any().any()}")
        print(f"Has Inf: {np.isinf(merchant_data.select_dtypes(include=[np.number])).any().any()}")
        
        # Show inf columns
        if np.isinf(merchant_data.select_dtypes(include=[np.number])).any().any():
            inf_cols = merchant_data.select_dtypes(include=[np.number]).columns[
                np.isinf(merchant_data.select_dtypes(include=[np.number])).any()
            ]
            print(f"Columns with Inf: {inf_cols.tolist()}")

In [ ]:
# Check for and handle missing values
print("=== DATA QUALITY CHECK ===\n")

for name, df in [('Train', df_train), ('Validate', df_validate), ('Test', df_test)]:
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"{name} dataset - Missing values:")
        print(null_counts[null_counts > 0])
        print()

# Fill NaN values with 0 (for cost_type percentages, NaN means 0% of that type)
print("Filling NaN values with 0...")
df_train = df_train.fillna(0)
df_validate = df_validate.fillna(0)
df_test = df_test.fillna(0)

# Check for and handle infinite values
print("\nChecking for infinite values...")
for name, df in [('Train', df_train), ('Validate', df_validate), ('Test', df_test)]:
    inf_mask = np.isinf(df.select_dtypes(include=[np.number])).any(axis=1)
    if inf_mask.sum() > 0:
        print(f"{name}: {inf_mask.sum()} rows with infinite values")

# Replace infinite values with 0
df_train = df_train.replace([np.inf, -np.inf], 0)
df_validate = df_validate.replace([np.inf, -np.inf], 0)
df_test = df_test.replace([np.inf, -np.inf], 0)

print("\nData cleaned successfully!")
print(f"Train: {df_train.shape}, Validate: {df_validate.shape}, Test: {df_test.shape}")

---
## 3. Advanced K-Means Hyperparameter Tuning

Beyond just testing different k values, we can optimize other K-Means hyperparameters:
- **init**: Initialization method ('k-means++' is smarter, 'random' is faster)
- **n_init**: Number of times algorithm runs with different centroid seeds (more is better but slower)
- **max_iter**: Maximum iterations (usually 300 is sufficient)

### 3.1 K-Means Hyperparameter Tuning - Raw Data

In [ ]:
# K-Means Comprehensive Hyperparameter Tuning - Raw Data
print("="*80)
print("K-MEANS COMPREHENSIVE HYPERPARAMETER TUNING - RAW DATA")
print("="*80)

# Define parameter grid
k_values = list(range(2, 11))  # Test k from 2 to 10
init_methods = ['k-means++', 'random']
n_init_values = [10, 20, 50]  # Number of random initializations

kmeans_tuning_results = []

print("\nTesting combinations of k, init method, and n_init...")
print("(This may take a moment)\n")

for k in k_values:
    for init_method in init_methods:
        for n_init in n_init_values:
            # Fit K-Means
            kmeans_test = KMeans(n_clusters=k, init=init_method, n_init=n_init, random_state=42, max_iter=300)
            train_labels = kmeans_test.fit_predict(X_train_scaled)
            
            # Predict on validation set
            validate_labels = kmeans_test.predict(X_validate_scaled)
            
            # Calculate metrics
            train_sil = silhouette_score(X_train_scaled, train_labels)
            train_db = davies_bouldin_score(X_train_scaled, train_labels)
            train_inertia = kmeans_test.inertia_
            
            validate_sil = silhouette_score(X_validate_scaled, validate_labels)
            validate_db = davies_bouldin_score(X_validate_scaled, validate_labels)
            
            kmeans_tuning_results.append({
                'k': k,
                'init': init_method,
                'n_init': n_init,
                'train_silhouette': train_sil,
                'train_davies_bouldin': train_db,
                'train_inertia': train_inertia,
                'validate_silhouette': validate_sil,
                'validate_davies_bouldin': validate_db
            })

# Convert to DataFrame
kmeans_tuning_df = pd.DataFrame(kmeans_tuning_results)

print("="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Find best parameters based on validation silhouette score
best_kmeans = kmeans_tuning_df.loc[kmeans_tuning_df['validate_silhouette'].idxmax()]

print(f"\nBest Parameters (by Validation Silhouette Score):")
print(f"  k = {int(best_kmeans['k'])}")
print(f"  init = '{best_kmeans['init']}'")
print(f"  n_init = {int(best_kmeans['n_init'])}")
print(f"\nExpected Performance:")
print(f"  Train Silhouette: {best_kmeans['train_silhouette']:.4f}")
print(f"  Train Davies-Bouldin: {best_kmeans['train_davies_bouldin']:.4f}")
print(f"  Validate Silhouette: {best_kmeans['validate_silhouette']:.4f}")
print(f"  Validate Davies-Bouldin: {best_kmeans['validate_davies_bouldin']:.4f}")

# Show top 10 configurations
print(f"\n{'='*80}")
print("TOP 10 CONFIGURATIONS (by Validation Silhouette Score):")
print(f"{'='*80}")
top_kmeans = kmeans_tuning_df.nlargest(10, 'validate_silhouette')[['k', 'init', 'n_init', 'train_silhouette', 'validate_silhouette', 'train_davies_bouldin', 'validate_davies_bouldin']]
print(top_kmeans.to_string(index=False))

# Analysis by init method
print(f"\n{'='*80}")
print("PERFORMANCE COMPARISON BY INITIALIZATION METHOD:")
print(f"{'='*80}\n")
init_comparison = kmeans_tuning_df.groupby('init').agg({
    'validate_silhouette': ['mean', 'std', 'max'],
    'validate_davies_bouldin': ['mean', 'std', 'min']
}).round(4)
print(init_comparison)

In [ ]:
# Visualization of K-Means Hyperparameter Tuning - Raw Data
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('K-Means Hyperparameter Tuning Results - Raw Data', fontsize=16, y=0.995)

# For each init method, create a heatmap
for idx, init_method in enumerate(['k-means++', 'random']):
    # Filter data for this init method
    data_subset = kmeans_tuning_df[kmeans_tuning_df['init'] == init_method]
    
    # Create pivot tables
    sil_pivot = data_subset.pivot(index='n_init', columns='k', values='validate_silhouette')
    db_pivot = data_subset.pivot(index='n_init', columns='k', values='validate_davies_bouldin')
    inertia_pivot = data_subset.pivot(index='n_init', columns='k', values='train_inertia')
    
    # Silhouette Score
    sns.heatmap(sil_pivot, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[idx, 0], 
                cbar_kws={'label': 'Silhouette Score'}, vmin=0, vmax=0.8)
    axes[idx, 0].set_title(f'Validation Silhouette Score\n(init={init_method})')
    axes[idx, 0].set_xlabel('Number of Clusters (k)')
    axes[idx, 0].set_ylabel('n_init')
    
    # Davies-Bouldin Index (lower is better, so reverse colormap)
    sns.heatmap(db_pivot, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=axes[idx, 1],
                cbar_kws={'label': 'Davies-Bouldin Index'})
    axes[idx, 1].set_title(f'Validation Davies-Bouldin Index\n(init={init_method})')
    axes[idx, 1].set_xlabel('Number of Clusters (k)')
    axes[idx, 1].set_ylabel('n_init')
    
    # Inertia
    sns.heatmap(inertia_pivot, annot=True, fmt='.0f', cmap='viridis', ax=axes[idx, 2],
                cbar_kws={'label': 'Inertia'})
    axes[idx, 2].set_title(f'Training Inertia\n(init={init_method})')
    axes[idx, 2].set_xlabel('Number of Clusters (k)')
    axes[idx, 2].set_ylabel('n_init')

plt.tight_layout()
plt.show()

# Comparison plot: Init method effect on performance
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for init_method in ['k-means++', 'random']:
    data_subset = kmeans_tuning_df[kmeans_tuning_df['init'] == init_method]
    # Average across n_init values for each k
    avg_by_k = data_subset.groupby('k').agg({
        'validate_silhouette': ['mean', 'std'],
        'validate_davies_bouldin': ['mean', 'std']
    })
    
    axes[0].plot(avg_by_k.index, avg_by_k[('validate_silhouette', 'mean')], 
                marker='o', label=init_method, linewidth=2)
    axes[0].fill_between(avg_by_k.index, 
                         avg_by_k[('validate_silhouette', 'mean')] - avg_by_k[('validate_silhouette', 'std')],
                         avg_by_k[('validate_silhouette', 'mean')] + avg_by_k[('validate_silhouette', 'std')],
                         alpha=0.2)
    
    axes[1].plot(avg_by_k.index, avg_by_k[('validate_davies_bouldin', 'mean')], 
                marker='o', label=init_method, linewidth=2)
    axes[1].fill_between(avg_by_k.index, 
                         avg_by_k[('validate_davies_bouldin', 'mean')] - avg_by_k[('validate_davies_bouldin', 'std')],
                         avg_by_k[('validate_davies_bouldin', 'mean')] + avg_by_k[('validate_davies_bouldin', 'std')],
                         alpha=0.2)

axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Validation Silhouette Score', fontsize=12)
axes[0].set_title('Effect of Init Method on Silhouette Score', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Validation Davies-Bouldin Index', fontsize=12)
axes[1].set_title('Effect of Init Method on Davies-Bouldin Index', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# compare both results

print("=== COMPARISON OF CLUSTERING RESULTS ===\n")
print(f"Original Data Clustering (k={optimal_k}):")
print(f"  Train Silhouette Score: {train_silhouette:.4f}")
print(f"  Train Davies-Bouldin Index: {train_davies_bouldin:.4f}")
print(f"  Validate Silhouette Score: {validate_silhouette:.4f}")
print(f"  Validate Davies-Bouldin Index: {validate_davies_bouldin:.4f}")
print(f"  Test Silhouette Score: {test_silhouette:.4f}")
print(f"  Test Davies-Bouldin Index: {test_davies_bouldin:.4f}")

print(f"\nZ-Scored Data Clustering (k={optimal_k_z}):")
print(f"  Train Silhouette Score: {train_silhouette_z:.4f}")
print(f"  Train Davies-Bouldin Index: {train_davies_bouldin_z:.4f}")
print(f"  Validate Silhouette Score: {validate_silhouette_z:.4f}")
print(f"  Validate Davies-Bouldin Index: {validate_davies_bouldin_z:.4f}")
print(f"  Test Silhouette Score: {test_silhouette_z:.4f}")
print(f"  Test Davies-Bouldin Index: {test_davies_bouldin_z:.4f}")



In [ ]:
# K-Means Comprehensive Hyperparameter Tuning - Z-Scored Data
print("="*80)
print("K-MEANS COMPREHENSIVE HYPERPARAMETER TUNING - Z-SCORED DATA")
print("="*80)

# Define parameter grid
k_values_z = list(range(2, 11))
init_methods_z = ['k-means++', 'random']
n_init_values_z = [10, 20, 50]

kmeans_tuning_results_z = []

print("\nTesting combinations of k, init method, and n_init...")
print("(This may take a moment)\n")

for k in k_values_z:
    for init_method in init_methods_z:
        for n_init in n_init_values_z:
            # Fit K-Means
            kmeans_test_z = KMeans(n_clusters=k, init=init_method, n_init=n_init, random_state=42, max_iter=300)
            train_labels_z = kmeans_test_z.fit_predict(X_train_z)
            
            # Predict on validation set
            validate_labels_z = kmeans_test_z.predict(X_validate_z)
            
            # Calculate metrics
            train_sil_z = silhouette_score(X_train_z, train_labels_z)
            train_db_z = davies_bouldin_score(X_train_z, train_labels_z)
            train_inertia_z = kmeans_test_z.inertia_
            
            validate_sil_z = silhouette_score(X_validate_z, validate_labels_z)
            validate_db_z = davies_bouldin_score(X_validate_z, validate_labels_z)
            
            kmeans_tuning_results_z.append({
                'k': k,
                'init': init_method,
                'n_init': n_init,
                'train_silhouette': train_sil_z,
                'train_davies_bouldin': train_db_z,
                'train_inertia': train_inertia_z,
                'validate_silhouette': validate_sil_z,
                'validate_davies_bouldin': validate_db_z
            })

# Convert to DataFrame
kmeans_tuning_df_z = pd.DataFrame(kmeans_tuning_results_z)

print("="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Find best parameters
best_kmeans_z = kmeans_tuning_df_z.loc[kmeans_tuning_df_z['validate_silhouette'].idxmax()]

print(f"\nBest Parameters (by Validation Silhouette Score):")
print(f"  k = {int(best_kmeans_z['k'])}")
print(f"  init = '{best_kmeans_z['init']}'")
print(f"  n_init = {int(best_kmeans_z['n_init'])}")
print(f"\nExpected Performance:")
print(f"  Train Silhouette: {best_kmeans_z['train_silhouette']:.4f}")
print(f"  Train Davies-Bouldin: {best_kmeans_z['train_davies_bouldin']:.4f}")
print(f"  Validate Silhouette: {best_kmeans_z['validate_silhouette']:.4f}")
print(f"  Validate Davies-Bouldin: {best_kmeans_z['validate_davies_bouldin']:.4f}")

# Show top 10 configurations
print(f"\n{'='*80}")
print("TOP 10 CONFIGURATIONS (by Validation Silhouette Score):")
print(f"{'='*80}")
top_kmeans_z = kmeans_tuning_df_z.nlargest(10, 'validate_silhouette')[['k', 'init', 'n_init', 'train_silhouette', 'validate_silhouette', 'train_davies_bouldin', 'validate_davies_bouldin']]
print(top_kmeans_z.to_string(index=False))

# Analysis by init method
print(f"\n{'='*80}")
print("PERFORMANCE COMPARISON BY INITIALIZATION METHOD:")
print(f"{'='*80}\n")
init_comparison_z = kmeans_tuning_df_z.groupby('init').agg({
    'validate_silhouette': ['mean', 'std', 'max'],
    'validate_davies_bouldin': ['mean', 'std', 'min']
}).round(4)
print(init_comparison_z)

In [ ]:
# Visualization of K-Means Hyperparameter Tuning - Z-Scored Data
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('K-Means Hyperparameter Tuning Results - Z-Scored Data', fontsize=16, y=0.995)

# For each init method, create a heatmap
for idx, init_method in enumerate(['k-means++', 'random']):
    # Filter data for this init method
    data_subset_z = kmeans_tuning_df_z[kmeans_tuning_df_z['init'] == init_method]
    
    # Create pivot tables
    sil_pivot_z = data_subset_z.pivot(index='n_init', columns='k', values='validate_silhouette')
    db_pivot_z = data_subset_z.pivot(index='n_init', columns='k', values='validate_davies_bouldin')
    inertia_pivot_z = data_subset_z.pivot(index='n_init', columns='k', values='train_inertia')
    
    # Silhouette Score
    sns.heatmap(sil_pivot_z, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[idx, 0], 
                cbar_kws={'label': 'Silhouette Score'}, vmin=0, vmax=0.8)
    axes[idx, 0].set_title(f'Validation Silhouette Score\n(init={init_method})')
    axes[idx, 0].set_xlabel('Number of Clusters (k)')
    axes[idx, 0].set_ylabel('n_init')
    
    # Davies-Bouldin Index (lower is better, so reverse colormap)
    sns.heatmap(db_pivot_z, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=axes[idx, 1],
                cbar_kws={'label': 'Davies-Bouldin Index'})
    axes[idx, 1].set_title(f'Validation Davies-Bouldin Index\n(init={init_method})')
    axes[idx, 1].set_xlabel('Number of Clusters (k)')
    axes[idx, 1].set_ylabel('n_init')
    
    # Inertia
    sns.heatmap(inertia_pivot_z, annot=True, fmt='.0f', cmap='viridis', ax=axes[idx, 2],
                cbar_kws={'label': 'Inertia'})
    axes[idx, 2].set_title(f'Training Inertia\n(init={init_method})')
    axes[idx, 2].set_xlabel('Number of Clusters (k)')
    axes[idx, 2].set_ylabel('n_init')

plt.tight_layout()
plt.show()

# Comparison plot: Init method effect on performance
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for init_method in ['k-means++', 'random']:
    data_subset_z = kmeans_tuning_df_z[kmeans_tuning_df_z['init'] == init_method]
    # Average across n_init values for each k
    avg_by_k_z = data_subset_z.groupby('k').agg({
        'validate_silhouette': ['mean', 'std'],
        'validate_davies_bouldin': ['mean', 'std']
    })
    
    axes[0].plot(avg_by_k_z.index, avg_by_k_z[('validate_silhouette', 'mean')], 
                marker='o', label=init_method, linewidth=2)
    axes[0].fill_between(avg_by_k_z.index, 
                         avg_by_k_z[('validate_silhouette', 'mean')] - avg_by_k_z[('validate_silhouette', 'std')],
                         avg_by_k_z[('validate_silhouette', 'mean')] + avg_by_k_z[('validate_silhouette', 'std')],
                         alpha=0.2)
    
    axes[1].plot(avg_by_k_z.index, avg_by_k_z[('validate_davies_bouldin', 'mean')], 
                marker='o', label=init_method, linewidth=2)
    axes[1].fill_between(avg_by_k_z.index, 
                         avg_by_k_z[('validate_davies_bouldin', 'mean')] - avg_by_k_z[('validate_davies_bouldin', 'std')],
                         avg_by_k_z[('validate_davies_bouldin', 'mean')] + avg_by_k_z[('validate_davies_bouldin', 'std')],
                         alpha=0.2)

axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Validation Silhouette Score', fontsize=12)
axes[0].set_title('Effect of Init Method on Silhouette Score (Z-Scored)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Validation Davies-Bouldin Index', fontsize=12)
axes[1].set_title('Effect of Init Method on Davies-Bouldin Index (Z-Scored)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 4. Advanced Hierarchical Clustering Hyperparameter Tuning

Hierarchical clustering has two main hyperparameters:
- **Linkage Method**: How to calculate distance between clusters (ward, complete, average, single)
- **Number of Clusters (n_clusters)**: Where to cut the dendrogram

We'll test all combinations comprehensively for both raw and z-scored data.

### 4.1 Hierarchical Hyperparameter Tuning - Raw Data

In [ ]:
# Hierarchical Clustering Comprehensive Hyperparameter Tuning - Raw Data
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist

print("="*80)
print("HIERARCHICAL CLUSTERING COMPREHENSIVE HYPERPARAMETER TUNING - RAW DATA")
print("="*80)

# Define parameter grid
linkage_methods = ['ward', 'complete', 'average', 'single']
n_clusters_range = list(range(2, 11))

hierarchical_tuning_results = []

print("\nTesting combinations of linkage methods and n_clusters...")
print("(This may take a moment)\n")

for linkage_method in linkage_methods:
    # Compute linkage matrix
    print(f"Computing {linkage_method} linkage...")
    linkage_matrix = linkage(X_train_scaled, method=linkage_method)
    
    for n_clusters in n_clusters_range:
        # Cut dendrogram at specific n_clusters
        cluster_labels_train = fcluster(linkage_matrix, n_clusters, criterion='maxclust') - 1
        
        # Compute cluster centers for prediction
        cluster_centers = []
        for i in range(n_clusters):
            cluster_mask = cluster_labels_train == i
            if np.sum(cluster_mask) > 0:  # Check if cluster has points
                center = X_train_scaled[cluster_mask].mean(axis=0)
                cluster_centers.append(center)
        
        if len(cluster_centers) < n_clusters:
            print(f"  Warning: {linkage_method} with n_clusters={n_clusters} produced {len(cluster_centers)} clusters")
            continue
            
        cluster_centers = np.array(cluster_centers)
        
        # Assign validation points to nearest cluster center
        cluster_labels_validate = cdist(X_validate_scaled, cluster_centers).argmin(axis=1)
        
        # Check if we have at least 2 unique clusters in both train and validate
        n_train_clusters = len(np.unique(cluster_labels_train))
        n_validate_clusters = len(np.unique(cluster_labels_validate))
        
        if n_train_clusters < 2 or n_validate_clusters < 2:
            print(f"  Skipping: {linkage_method} with n_clusters={n_clusters} - insufficient clusters (train={n_train_clusters}, val={n_validate_clusters})")
            continue
        
        # Calculate metrics
        train_sil = silhouette_score(X_train_scaled, cluster_labels_train)
        train_db = davies_bouldin_score(X_train_scaled, cluster_labels_train)
        
        validate_sil = silhouette_score(X_validate_scaled, cluster_labels_validate)
        validate_db = davies_bouldin_score(X_validate_scaled, cluster_labels_validate)
        
        hierarchical_tuning_results.append({
            'linkage': linkage_method,
            'n_clusters': n_clusters,
            'train_silhouette': train_sil,
            'train_davies_bouldin': train_db,
            'validate_silhouette': validate_sil,
            'validate_davies_bouldin': validate_db
        })

# Convert to DataFrame
hierarchical_tuning_df = pd.DataFrame(hierarchical_tuning_results)

print("\n" + "="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Find best parameters
best_hierarchical = hierarchical_tuning_df.loc[hierarchical_tuning_df['validate_silhouette'].idxmax()]

print(f"\nBest Parameters (by Validation Silhouette Score):")
print(f"  Linkage Method = '{best_hierarchical['linkage']}'")
print(f"  n_clusters = {int(best_hierarchical['n_clusters'])}")
print(f"\nExpected Performance:")
print(f"  Train Silhouette: {best_hierarchical['train_silhouette']:.4f}")
print(f"  Train Davies-Bouldin: {best_hierarchical['train_davies_bouldin']:.4f}")
print(f"  Validate Silhouette: {best_hierarchical['validate_silhouette']:.4f}")
print(f"  Validate Davies-Bouldin: {best_hierarchical['validate_davies_bouldin']:.4f}")

# Show top 10 configurations
print(f"\n{'='*80}")
print("TOP 10 CONFIGURATIONS (by Validation Silhouette Score):")
print(f"{'='*80}")
top_hierarchical = hierarchical_tuning_df.nlargest(10, 'validate_silhouette')[['linkage', 'n_clusters', 'train_silhouette', 'validate_silhouette', 'train_davies_bouldin', 'validate_davies_bouldin']]
print(top_hierarchical.to_string(index=False))

# Analysis by linkage method
print(f"\n{'='*80}")
print("PERFORMANCE COMPARISON BY LINKAGE METHOD:")
print(f"{'='*80}\n")
linkage_comparison = hierarchical_tuning_df.groupby('linkage').agg({
    'validate_silhouette': ['mean', 'std', 'max'],
    'validate_davies_bouldin': ['mean', 'std', 'min']
}).round(4)
print(linkage_comparison)

In [ ]:
# Visualization of Hierarchical Hyperparameter Tuning - Raw Data
fig, axes = plt.subplots(1, 4, figsize=(22, 5))
fig.suptitle('Hierarchical Clustering Hyperparameter Tuning Results - Raw Data', fontsize=16, y=1.02)

for idx, linkage_method in enumerate(linkage_methods):
    # Filter data for this linkage method
    data_subset = hierarchical_tuning_df[hierarchical_tuning_df['linkage'] == linkage_method]
    
    # Plot Silhouette Score
    axes[idx].plot(data_subset['n_clusters'], data_subset['validate_silhouette'], 
                   marker='o', linewidth=2, markersize=8, label='Silhouette (higher better)')
    axes[idx].set_xlabel('Number of Clusters', fontsize=11)
    axes[idx].set_ylabel('Validation Silhouette Score', fontsize=11)
    axes[idx].set_title(f'{linkage_method.capitalize()} Linkage', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xticks(n_clusters_range)
    
    # Add Davies-Bouldin on secondary axis (inverted scale)
    ax2 = axes[idx].twinx()
    ax2.plot(data_subset['n_clusters'], data_subset['validate_davies_bouldin'], 
            marker='s', color='orange', linewidth=2, markersize=8, 
            label='Davies-Bouldin (lower better)', alpha=0.7)
    ax2.set_ylabel('Validation Davies-Bouldin Index', fontsize=11, color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')
    
    # Combine legends
    lines1, labels1 = axes[idx].get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    axes[idx].legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=9)

plt.tight_layout()
plt.show()

# Comparison heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Hierarchical Clustering: Linkage Method Comparison Heatmap - Raw Data', fontsize=15)

# Silhouette Score heatmap
sil_pivot = hierarchical_tuning_df.pivot(index='linkage', columns='n_clusters', values='validate_silhouette')
sns.heatmap(sil_pivot, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[0], 
            cbar_kws={'label': 'Silhouette Score'}, vmin=0, vmax=1)
axes[0].set_title('Validation Silhouette Score', fontsize=13)
axes[0].set_xlabel('Number of Clusters')
axes[0].set_ylabel('Linkage Method')

# Davies-Bouldin Index heatmap (lower is better, so reverse colormap)
db_pivot = hierarchical_tuning_df.pivot(index='linkage', columns='n_clusters', values='validate_davies_bouldin')
sns.heatmap(db_pivot, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=axes[1],
            cbar_kws={'label': 'Davies-Bouldin Index'})
axes[1].set_title('Validation Davies-Bouldin Index', fontsize=13)
axes[1].set_xlabel('Number of Clusters')
axes[1].set_ylabel('Linkage Method')

plt.tight_layout()
plt.show()

### 7.2 Hierarchical Hyperparameter Tuning - Z-Scored Data

In [ ]:
# Hierarchical Clustering Comprehensive Hyperparameter Tuning - Z-Scored Data
print("="*80)
print("HIERARCHICAL CLUSTERING COMPREHENSIVE HYPERPARAMETER TUNING - Z-SCORED DATA")
print("="*80)

# Define parameter grid
linkage_methods_z = ['ward', 'complete', 'average', 'single']
n_clusters_range_z = list(range(2, 11))

hierarchical_tuning_results_z = []

print("\nTesting combinations of linkage methods and n_clusters...")
print("(This may take a moment)\n")

for linkage_method in linkage_methods_z:
    # Compute linkage matrix
    print(f"Computing {linkage_method} linkage...")
    linkage_matrix_z = linkage(X_train_z, method=linkage_method)
    
    for n_clusters in n_clusters_range_z:
        # Cut dendrogram at specific n_clusters
        cluster_labels_train_z = fcluster(linkage_matrix_z, n_clusters, criterion='maxclust') - 1
        
        # Compute cluster centers for prediction
        cluster_centers_z = []
        for i in range(n_clusters):
            cluster_mask = cluster_labels_train_z == i
            if np.sum(cluster_mask) > 0:  # Check if cluster has points
                center = X_train_z[cluster_mask].mean(axis=0)
                cluster_centers_z.append(center)
        
        if len(cluster_centers_z) < n_clusters:
            print(f"  Warning: {linkage_method} with n_clusters={n_clusters} produced {len(cluster_centers_z)} clusters")
            continue
            
        cluster_centers_z = np.array(cluster_centers_z)
        
        # Assign validation points to nearest cluster center
        cluster_labels_validate_z = cdist(X_validate_z, cluster_centers_z).argmin(axis=1)
        
        # Check if we have at least 2 unique clusters in both train and validate
        n_train_clusters_z = len(np.unique(cluster_labels_train_z))
        n_validate_clusters_z = len(np.unique(cluster_labels_validate_z))
        
        if n_train_clusters_z < 2 or n_validate_clusters_z < 2:
            print(f"  Skipping: {linkage_method} with n_clusters={n_clusters} - insufficient clusters (train={n_train_clusters_z}, val={n_validate_clusters_z})")
            continue
        
        # Calculate metrics
        train_sil_z = silhouette_score(X_train_z, cluster_labels_train_z)
        train_db_z = davies_bouldin_score(X_train_z, cluster_labels_train_z)
        
        validate_sil_z = silhouette_score(X_validate_z, cluster_labels_validate_z)
        validate_db_z = davies_bouldin_score(X_validate_z, cluster_labels_validate_z)
        
        hierarchical_tuning_results_z.append({
            'linkage': linkage_method,
            'n_clusters': n_clusters,
            'train_silhouette': train_sil_z,
            'train_davies_bouldin': train_db_z,
            'validate_silhouette': validate_sil_z,
            'validate_davies_bouldin': validate_db_z
        })

# Convert to DataFrame
hierarchical_tuning_df_z = pd.DataFrame(hierarchical_tuning_results_z)

print("\n" + "="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Find best parameters
best_hierarchical_z = hierarchical_tuning_df_z.loc[hierarchical_tuning_df_z['validate_silhouette'].idxmax()]

print(f"\nBest Parameters (by Validation Silhouette Score):")
print(f"  Linkage Method = '{best_hierarchical_z['linkage']}'")
print(f"  n_clusters = {int(best_hierarchical_z['n_clusters'])}")
print(f"\nExpected Performance:")
print(f"  Train Silhouette: {best_hierarchical_z['train_silhouette']:.4f}")
print(f"  Train Davies-Bouldin: {best_hierarchical_z['train_davies_bouldin']:.4f}")
print(f"  Validate Silhouette: {best_hierarchical_z['validate_silhouette']:.4f}")
print(f"  Validate Davies-Bouldin: {best_hierarchical_z['validate_davies_bouldin']:.4f}")

# Show top 10 configurations
print(f"\n{'='*80}")
print("TOP 10 CONFIGURATIONS (by Validation Silhouette Score):")
print(f"{'='*80}")
top_hierarchical_z = hierarchical_tuning_df_z.nlargest(10, 'validate_silhouette')[['linkage', 'n_clusters', 'train_silhouette', 'validate_silhouette', 'train_davies_bouldin', 'validate_davies_bouldin']]
print(top_hierarchical_z.to_string(index=False))

# Analysis by linkage method
print(f"\n{'='*80}")
print("PERFORMANCE COMPARISON BY LINKAGE METHOD:")
print(f"{'='*80}\n")
linkage_comparison_z = hierarchical_tuning_df_z.groupby('linkage').agg({
    'validate_silhouette': ['mean', 'std', 'max'],
    'validate_davies_bouldin': ['mean', 'std', 'min']
}).round(4)
print(linkage_comparison_z)

In [ ]:
# Visualization of Hierarchical Hyperparameter Tuning - Z-Scored Data
fig, axes = plt.subplots(1, 4, figsize=(22, 5))
fig.suptitle('Hierarchical Clustering Hyperparameter Tuning Results - Z-Scored Data', fontsize=16, y=1.02)

for idx, linkage_method in enumerate(linkage_methods_z):
    # Filter data for this linkage method
    data_subset_z = hierarchical_tuning_df_z[hierarchical_tuning_df_z['linkage'] == linkage_method]
    
    # Plot Silhouette Score
    axes[idx].plot(data_subset_z['n_clusters'], data_subset_z['validate_silhouette'], 
                   marker='o', linewidth=2, markersize=8, label='Silhouette (higher better)')
    axes[idx].set_xlabel('Number of Clusters', fontsize=11)
    axes[idx].set_ylabel('Validation Silhouette Score', fontsize=11)
    axes[idx].set_title(f'{linkage_method.capitalize()} Linkage', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xticks(n_clusters_range_z)
    
    # Add Davies-Bouldin on secondary axis (inverted scale)
    ax2 = axes[idx].twinx()
    ax2.plot(data_subset_z['n_clusters'], data_subset_z['validate_davies_bouldin'], 
            marker='s', color='orange', linewidth=2, markersize=8, 
            label='Davies-Bouldin (lower better)', alpha=0.7)
    ax2.set_ylabel('Validation Davies-Bouldin Index', fontsize=11, color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')
    
    # Combine legends
    lines1, labels1 = axes[idx].get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    axes[idx].legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=9)

plt.tight_layout()
plt.show()

# Comparison heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Hierarchical Clustering: Linkage Method Comparison Heatmap - Z-Scored Data', fontsize=15)

# Silhouette Score heatmap
sil_pivot_z = hierarchical_tuning_df_z.pivot(index='linkage', columns='n_clusters', values='validate_silhouette')
sns.heatmap(sil_pivot_z, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[0], 
            cbar_kws={'label': 'Silhouette Score'}, vmin=0, vmax=1)
axes[0].set_title('Validation Silhouette Score', fontsize=13)
axes[0].set_xlabel('Number of Clusters')
axes[0].set_ylabel('Linkage Method')

# Davies-Bouldin Index heatmap (lower is better, so reverse colormap)
db_pivot_z = hierarchical_tuning_df_z.pivot(index='linkage', columns='n_clusters', values='validate_davies_bouldin')
sns.heatmap(db_pivot_z, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=axes[1],
            cbar_kws={'label': 'Davies-Bouldin Index'})
axes[1].set_title('Validation Davies-Bouldin Index', fontsize=13)
axes[1].set_xlabel('Number of Clusters')
axes[1].set_ylabel('Linkage Method')

plt.tight_layout()
plt.show()

### 6.2 K-Means Hyperparameter Tuning - Z-Scored Data

In [ ]:
# show plots for both sets of data side by side for comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
datasets = [
    (X_train_scaled, df_train['cluster'], 'Train - Original'),
    (X_validate_scaled, df_validate['cluster'], 'Validate - Original'),
    (X_test_scaled, df_test['cluster'], 'Test - Original'),
    (X_train_z, df_train_z['cluster'], 'Train - Z-Scored'),
    (X_validate_z, df_validate_z['cluster'], 'Validate - Z-Scored'),
    (X_test_z, df_test_z['cluster'], 'Test - Z-Scored')
]

for ax, (X, clusters, title) in zip(axes.flatten(), datasets):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
    if 'Z-Scored' in title:
        ax.scatter(kmeans_z.cluster_centers_[:, 0], kmeans_z.cluster_centers_[:, 1], 
                   c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroids')
    else:
        ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                   c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroids')
    ax.set_xlabel(f'Feature 1: {feature_cols[0]}')
    ax.set_ylabel(f'Feature 2: {feature_cols[1]}')
    ax.set_title(title)
    ax.legend()
    plt.colorbar(scatter, ax=ax, label='Cluster')
    
plt.tight_layout()
plt.show()

---
## 5. K-Means Clustering on Raw Data

### 5.1 Data Preparation


In [ ]:
# K Means clustering on base metrics
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Prepare data - exclude merchant_id column for clustering
feature_cols = [col for col in df_train.columns if col != 'merchant_id']
X_train_scaled = df_train[feature_cols].values
X_validate_scaled = df_validate[feature_cols].values
X_test_scaled = df_test[feature_cols].values

print(f"Training data shape: {X_train_scaled.shape}")
print(f"Features used: {feature_cols}")
print(f"Number of merchants - Train: {len(df_train)}, Validate: {len(df_validate)}, Test: {len(df_test)}")
print(f"\nUsing raw data without scaling")


### 3.2 Elbow Method & Silhouette Analysis (k=2-10)


In [ ]:
# Elbow Method - Find optimal number of clusters
inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_train_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_train_scaled, kmeans.labels_))

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
ax1.plot(k_range, inertias, 'bo-')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Within-cluster sum of squares)')
ax1.set_title('Elbow Method For Optimal k')
ax1.grid(True)

# Silhouette plot
ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score vs Number of Clusters')
ax2.grid(True)

plt.tight_layout()
plt.show()

print("\nSilhouette Scores:")
for k, score in zip(k_range, silhouette_scores):
    print(f"k={k}: {score:.4f}")


### 3.3 Fit Model with Optimal k (Using Tuned Hyperparameters)

**Note**: Run Section 6.1 (K-Means Hyperparameter Tuning) first to populate optimal parameters.

In [ ]:
# Fit K-Means with optimal hyperparameters from tuning (Section 6.1)
# If best_kmeans doesn't exist, run Section 6.1 first
try:
    optimal_k = int(best_kmeans['k'])
    optimal_init = best_kmeans['init']
    optimal_n_init = int(best_kmeans['n_init'])
    print(f"Using tuned parameters: k={optimal_k}, init='{optimal_init}', n_init={optimal_n_init}")
except (NameError, KeyError):
    print("Warning: best_kmeans not found. Run Section 6.1 first or using fallback k=5")
    optimal_k = 5
    optimal_init = 'k-means++'
    optimal_n_init = 10

kmeans = KMeans(n_clusters=optimal_k, init=optimal_init, n_init=optimal_n_init, random_state=42, max_iter=300)
kmeans.fit(X_train_scaled)

# Get cluster labels
df_train['cluster'] = kmeans.labels_
df_validate['cluster'] = kmeans.predict(X_validate_scaled)
df_test['cluster'] = kmeans.predict(X_test_scaled)

# Evaluation metrics
train_silhouette = silhouette_score(X_train_scaled, df_train['cluster'])
train_davies_bouldin = davies_bouldin_score(X_train_scaled, df_train['cluster'])

print(f"\nK-Means with {optimal_k} clusters")
print(f"Training Set Metrics:")
print(f"  Silhouette Score: {train_silhouette:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {train_davies_bouldin:.4f} (lower is better)")
print(f"  Inertia: {kmeans.inertia_:.2f}")

print(f"\nCluster Distribution (Train):")
print(df_train['cluster'].value_counts().sort_index())

In [ ]:
# Visualize clusters (2D projection using first 2 features)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets = [
    (X_train_scaled, df_train['cluster'], 'Train'),
    (X_validate_scaled, df_validate['cluster'], 'Validate'),
    (X_test_scaled, df_test['cluster'], 'Test')
]

for ax, (X, clusters, title) in zip(axes, datasets):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
    ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroids')
    ax.set_xlabel(f'Feature 1: {feature_cols[0]}')
    ax.set_ylabel(f'Feature 2: {feature_cols[1]}')
    ax.set_title(f'{title} Set - {optimal_k} Clusters')
    ax.legend()
    plt.colorbar(scatter, ax=ax, label='Cluster')

plt.tight_layout()
plt.show()


### 3.4 Cluster Analysis & Statistics


In [ ]:
# Analyze cluster characteristics
print("=== CLUSTER CHARACTERISTICS ===\n")

# Use raw data (no scaling was applied)
df_train_with_features = pd.DataFrame(X_train_scaled, columns=feature_cols)
df_train_with_features['cluster'] = df_train['cluster'].values

# Calculate statistics per cluster
cluster_stats = df_train_with_features.groupby('cluster').agg(['mean', 'std', 'count'])

print("Cluster Statistics (mean values):")
print(cluster_stats.xs('mean', level=1, axis=1))

print("\n\nCluster Sizes:")
print(df_train_with_features['cluster'].value_counts().sort_index())


In [ ]:
# score the clusters on validate and test sets
validate_silhouette = silhouette_score(X_validate_scaled, df_validate['cluster'])
validate_davies_bouldin = davies_bouldin_score(X_validate_scaled, df_validate['cluster'])

print(f"Validation Set Metrics:")
print(f"  Silhouette Score: {validate_silhouette:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {validate_davies_bouldin:.4f} (lower is better)")

test_silhouette = silhouette_score(X_test_scaled, df_test['cluster'])
test_davies_bouldin = davies_bouldin_score(X_test_scaled, df_test['cluster'])
print(f"\nTest Set Metrics:")
print(f"  Silhouette Score: {test_silhouette:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {test_davies_bouldin:.4f} (lower is better)")


In [ ]:
# Investigate the features in detail
print("=== FEATURE INVESTIGATION ===\n")

print("Column names in training data:")
print(df_train.columns.tolist())
print(f"\nShape: {df_train.shape}")

print("\n" + "="*60)
print("Feature columns used for clustering:")
print(feature_cols)

print("\n" + "="*60)
print("Sample of raw data (first 10 merchants):")
print(df_train.head(10))

print("\n" + "="*60)
print("Descriptive statistics for all features:")
print(df_train[feature_cols].describe())

print("\n" + "="*60)
print("Check for any negative values:")
for col in feature_cols:
    neg_count = (df_train[col] < 0).sum()
    if neg_count > 0:
        print(f"{col}: {neg_count} negative values (min: {df_train[col].min():.4f})")

print("\n" + "="*60)
print("Check for zero/non-zero patterns:")
for col in feature_cols:
    zero_count = (df_train[col] == 0).sum()
    nonzero_count = (df_train[col] != 0).sum()
    print(f"{col}: {zero_count} zeros, {nonzero_count} non-zeros")

---
## 6. K-Means Clustering on Z-Scored Data

Now clustering on the pre-z-scored datasets: `*_base_zscored.csv`

### 6.1 Load Z-Scored Data


In [ ]:
# Load z-scored datasets
df_train_z = pd.read_csv('base_features/df_train_base_zscored.csv')
df_validate_z = pd.read_csv('base_features/df_validate_base_zscored.csv')
df_test_z = pd.read_csv('base_features/df_test_base_zscored.csv')

print(f"Loaded z-scored datasets:")
print(f"  Train: {df_train_z.shape}")
print(f"  Validate: {df_validate_z.shape}")
print(f"  Test: {df_test_z.shape}")
print(f"\nColumns: {df_train_z.columns.tolist()}")

In [ ]:
# Check for and handle missing values
print("=== DATA QUALITY CHECK (Z-SCORED) ===\n")

for name, df in [('Train', df_train_z), ('Validate', df_validate_z), ('Test', df_test_z)]:
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"{name} dataset - Missing values:")
        print(null_counts[null_counts > 0])
        print()

# Fill NaN values with 0
print("Filling NaN values with 0...")
df_train_z = df_train_z.fillna(0)
df_validate_z = df_validate_z.fillna(0)
df_test_z = df_test_z.fillna(0)

# Check for and handle infinite values
print("\nChecking for infinite values...")
for name, df in [('Train', df_train_z), ('Validate', df_validate_z), ('Test', df_test_z)]:
    inf_mask = np.isinf(df.select_dtypes(include=[np.number])).any(axis=1)
    if inf_mask.sum() > 0:
        print(f"{name}: {inf_mask.sum()} rows with infinite values")

# Replace infinite values with 0
df_train_z = df_train_z.replace([np.inf, -np.inf], 0)
df_validate_z = df_validate_z.replace([np.inf, -np.inf], 0)
df_test_z = df_test_z.replace([np.inf, -np.inf], 0)

print("\nData cleaned successfully!")
print(f"Train: {df_train_z.shape}, Validate: {df_validate_z.shape}, Test: {df_test_z.shape}")

In [ ]:
# Prepare data - data is already z-scored, so no need to scale again
feature_cols_z = [col for col in df_train_z.columns if col != 'merchant_id']
X_train_z = df_train_z[feature_cols_z].values
X_validate_z = df_validate_z[feature_cols_z].values
X_test_z = df_test_z[feature_cols_z].values

print(f"Training data shape: {X_train_z.shape}")
print(f"Features used: {feature_cols_z}")
print(f"Number of merchants - Train: {len(df_train_z)}, Validate: {len(df_validate_z)}, Test: {len(df_test_z)}")
print(f"\nData is pre-z-scored - using directly without StandardScaler")

### 4.2 Elbow Method & Silhouette Analysis (k=2-10)


In [ ]:
# Elbow Method - Find optimal number of clusters (Z-scored data)
inertias_z = []
silhouette_scores_z = []
k_range_z = range(2, 11)

for k in k_range_z:
    kmeans_z = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_z.fit(X_train_z)
    inertias_z.append(kmeans_z.inertia_)
    silhouette_scores_z.append(silhouette_score(X_train_z, kmeans_z.labels_))

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
ax1.plot(k_range_z, inertias_z, 'bo-')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Within-cluster sum of squares)')
ax1.set_title('Elbow Method For Optimal k (Z-Scored Data)')
ax1.grid(True)

# Silhouette plot
ax2.plot(k_range_z, silhouette_scores_z, 'ro-')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score vs Number of Clusters (Z-Scored Data)')
ax2.grid(True)

plt.tight_layout()
plt.show()

print("\nSilhouette Scores (Z-Scored):")
for k, score in zip(k_range_z, silhouette_scores_z):
    print(f"k={k}: {score:.4f}")

### 4.3 Fit Model with Optimal k (Using Tuned Hyperparameters)

**Note**: Run Section 6.2 (K-Means Hyperparameter Tuning - Z-Scored) first to populate optimal parameters.

In [ ]:
# Fit K-Means with optimal hyperparameters from tuning (Section 6.2)
# If best_kmeans_z doesn't exist, run Section 6.2 first
try:
    optimal_k_z = int(best_kmeans_z['k'])
    optimal_init_z = best_kmeans_z['init']
    optimal_n_init_z = int(best_kmeans_z['n_init'])
    print(f"Using tuned parameters: k={optimal_k_z}, init='{optimal_init_z}', n_init={optimal_n_init_z}")
except (NameError, KeyError):
    print("Warning: best_kmeans_z not found. Run Section 6.2 first or using fallback k=5")
    optimal_k_z = 5
    optimal_init_z = 'k-means++'
    optimal_n_init_z = 10

kmeans_z = KMeans(n_clusters=optimal_k_z, init=optimal_init_z, n_init=optimal_n_init_z, random_state=42, max_iter=300)
kmeans_z.fit(X_train_z)

# Get cluster labels
df_train_z['cluster'] = kmeans_z.labels_
df_validate_z['cluster'] = kmeans_z.predict(X_validate_z)
df_test_z['cluster'] = kmeans_z.predict(X_test_z)

# Evaluation metrics
train_silhouette_z = silhouette_score(X_train_z, df_train_z['cluster'])
train_davies_bouldin_z = davies_bouldin_score(X_train_z, df_train_z['cluster'])

print(f"\nK-Means with {optimal_k_z} clusters (Z-Scored Data)")
print(f"Training Set Metrics:")
print(f"  Silhouette Score: {train_silhouette_z:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {train_davies_bouldin_z:.4f} (lower is better)")
print(f"  Inertia: {kmeans_z.inertia_:.2f}")

print(f"\nCluster Distribution (Train):")
print(df_train_z['cluster'].value_counts().sort_index())

In [ ]:
# Visualize clusters (Z-scored data)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets_z = [
    (X_train_z, df_train_z['cluster'], 'Train'),
    (X_validate_z, df_validate_z['cluster'], 'Validate'),
    (X_test_z, df_test_z['cluster'], 'Test')
]

for ax, (X, clusters, title) in zip(axes, datasets_z):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
    ax.scatter(kmeans_z.cluster_centers_[:, 0], kmeans_z.cluster_centers_[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroids')
    ax.set_xlabel(f'Feature 1: {feature_cols_z[0]}')
    ax.set_ylabel(f'Feature 2: {feature_cols_z[1]}')
    ax.set_title(f'{title} Set - {optimal_k_z} Clusters (Z-Scored)')
    ax.legend()
    plt.colorbar(scatter, ax=ax, label='Cluster')

plt.tight_layout()
plt.show()

### 4.4 Cluster Analysis & Statistics


In [ ]:
# Analyze cluster characteristics (Z-scored data)
print("=== CLUSTER CHARACTERISTICS (Z-SCORED DATA) ===\n")

df_train_z_features = pd.DataFrame(X_train_z, columns=feature_cols_z)
df_train_z_features['cluster'] = df_train_z['cluster'].values
df_train_z_features['merchant_id'] = df_train_z['merchant_id'].values

# Calculate statistics per cluster
cluster_stats_z = df_train_z_features.groupby('cluster').agg(['mean', 'std', 'count'])

print("Cluster Statistics (mean values):")
print(cluster_stats_z.xs('mean', level=1, axis=1))

print("\n\nCluster Sizes:")
print(df_train_z_features['cluster'].value_counts().sort_index())

In [ ]:
# Score the clusters on validate and test sets (Z-scored data)
validate_silhouette_z = silhouette_score(X_validate_z, df_validate_z['cluster'])
validate_davies_bouldin_z = davies_bouldin_score(X_validate_z, df_validate_z['cluster'])

print(f"Validation Set Metrics (Z-Scored):")
print(f"  Silhouette Score: {validate_silhouette_z:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {validate_davies_bouldin_z:.4f} (lower is better)")

test_silhouette_z = silhouette_score(X_test_z, df_test_z['cluster'])
test_davies_bouldin_z = davies_bouldin_score(X_test_z, df_test_z['cluster'])
print(f"\nTest Set Metrics (Z-Scored):")
print(f"  Silhouette Score: {test_silhouette_z:.4f} (higher is better, range: -1 to 1)")
print(f"  Davies-Bouldin Index: {test_davies_bouldin_z:.4f} (lower is better)")

---
## 7. Extended K-Value Testing (k=2 to k=15)

Comprehensive evaluation of different k values to determine optimal cluster count.

### 7.1 Compute Metrics for Multiple k Values


In [ ]:
# Test for a wider range of k values with comprehensive metrics
# Testing k from 2 to 15 for both raw and z-scored data

k_range_extended = range(2, 16)

# Store results for raw data
results_raw = {
    'k': [],
    'train_silhouette': [],
    'train_davies_bouldin': [],
    'validate_silhouette': [],
    'validate_davies_bouldin': [],
    'inertia': []
}

# Store results for z-scored data
results_zscored = {
    'k': [],
    'train_silhouette': [],
    'train_davies_bouldin': [],
    'validate_silhouette': [],
    'validate_davies_bouldin': [],
    'inertia': []
}

print("Testing k values from 2 to 15...")
print("\n" + "="*80)

for k in k_range_extended:
    # Raw data clustering
    kmeans_test = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_test.fit(X_train_scaled)
    train_labels = kmeans_test.labels_
    validate_labels = kmeans_test.predict(X_validate_scaled)
    
    results_raw['k'].append(k)
    results_raw['inertia'].append(kmeans_test.inertia_)
    results_raw['train_silhouette'].append(silhouette_score(X_train_scaled, train_labels))
    results_raw['train_davies_bouldin'].append(davies_bouldin_score(X_train_scaled, train_labels))
    results_raw['validate_silhouette'].append(silhouette_score(X_validate_scaled, validate_labels))
    results_raw['validate_davies_bouldin'].append(davies_bouldin_score(X_validate_scaled, validate_labels))
    
    # Z-scored data clustering
    kmeans_test_z = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_test_z.fit(X_train_z)
    train_labels_z = kmeans_test_z.labels_
    validate_labels_z = kmeans_test_z.predict(X_validate_z)
    
    results_zscored['k'].append(k)
    results_zscored['inertia'].append(kmeans_test_z.inertia_)
    results_zscored['train_silhouette'].append(silhouette_score(X_train_z, train_labels_z))
    results_zscored['train_davies_bouldin'].append(davies_bouldin_score(X_train_z, train_labels_z))
    results_zscored['validate_silhouette'].append(silhouette_score(X_validate_z, validate_labels_z))
    results_zscored['validate_davies_bouldin'].append(davies_bouldin_score(X_validate_z, validate_labels_z))

print("Completed testing all k values!")
print("\n" + "="*80)
print("\nRAW DATA RESULTS:")
import pandas as pd
df_results_raw = pd.DataFrame(results_raw)
print(df_results_raw.to_string(index=False))

print("\n" + "="*80)
print("\nZ-SCORED DATA RESULTS:")
df_results_zscored = pd.DataFrame(results_zscored)
print(df_results_zscored.to_string(index=False))


### 6.2 Visualization & Optimal k Recommendations


In [ ]:
# Visualize metrics across different k values
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Raw data plots
axes[0, 0].plot(results_raw['k'], results_raw['inertia'], 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 0].set_ylabel('Inertia', fontsize=12)
axes[0, 0].set_title('Raw Data: Elbow Method', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(results_raw['k'], results_raw['train_silhouette'], 'go-', label='Train', linewidth=2, markersize=8)
axes[0, 1].plot(results_raw['k'], results_raw['validate_silhouette'], 'ro-', label='Validate', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 1].set_ylabel('Silhouette Score', fontsize=12)
axes[0, 1].set_title('Raw Data: Silhouette Score', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(results_raw['k'], results_raw['train_davies_bouldin'], 'go-', label='Train', linewidth=2, markersize=8)
axes[0, 2].plot(results_raw['k'], results_raw['validate_davies_bouldin'], 'ro-', label='Validate', linewidth=2, markersize=8)
axes[0, 2].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 2].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[0, 2].set_title('Raw Data: Davies-Bouldin Index (lower is better)', fontsize=14, fontweight='bold')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Z-scored data plots
axes[1, 0].plot(results_zscored['k'], results_zscored['inertia'], 'bo-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 0].set_ylabel('Inertia', fontsize=12)
axes[1, 0].set_title('Z-Scored Data: Elbow Method', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(results_zscored['k'], results_zscored['train_silhouette'], 'go-', label='Train', linewidth=2, markersize=8)
axes[1, 1].plot(results_zscored['k'], results_zscored['validate_silhouette'], 'ro-', label='Validate', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 1].set_ylabel('Silhouette Score', fontsize=12)
axes[1, 1].set_title('Z-Scored Data: Silhouette Score', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(results_zscored['k'], results_zscored['train_davies_bouldin'], 'go-', label='Train', linewidth=2, markersize=8)
axes[1, 2].plot(results_zscored['k'], results_zscored['validate_davies_bouldin'], 'ro-', label='Validate', linewidth=2, markersize=8)
axes[1, 2].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 2].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[1, 2].set_title('Z-Scored Data: Davies-Bouldin Index (lower is better)', fontsize=14, fontweight='bold')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal k based on different criteria
print("\n" + "="*80)
print("RECOMMENDED K VALUES:")
print("="*80)

# Raw data recommendations
best_k_raw_sil = results_raw['k'][results_raw['validate_silhouette'].index(max(results_raw['validate_silhouette']))]
best_k_raw_db = results_raw['k'][results_raw['validate_davies_bouldin'].index(min(results_raw['validate_davies_bouldin']))]
print(f"\nRaw Data:")
print(f"  Best k by Validation Silhouette Score: k={best_k_raw_sil} (score={max(results_raw['validate_silhouette']):.4f})")
print(f"  Best k by Validation Davies-Bouldin: k={best_k_raw_db} (score={min(results_raw['validate_davies_bouldin']):.4f})")

# Z-scored data recommendations
best_k_z_sil = results_zscored['k'][results_zscored['validate_silhouette'].index(max(results_zscored['validate_silhouette']))]
best_k_z_db = results_zscored['k'][results_zscored['validate_davies_bouldin'].index(min(results_zscored['validate_davies_bouldin']))]
print(f"\nZ-Scored Data:")
print(f"  Best k by Validation Silhouette Score: k={best_k_z_sil} (score={max(results_zscored['validate_silhouette']):.4f})")
print(f"  Best k by Validation Davies-Bouldin: k={best_k_z_db} (score={min(results_zscored['validate_davies_bouldin']):.4f})")


---
## 8. Hierarchical Clustering

Hierarchical clustering builds a tree of clusters (dendrogram) without requiring a pre-specified number of clusters.

### 8.1 Hierarchical Clustering on Raw Data


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist

# Perform hierarchical clustering with different linkage methods
linkage_methods = ['ward', 'complete', 'average']

# We'll use a sample of the data for dendrogram visualization (dendrograms get messy with too many points)
sample_size = 500
sample_indices = np.random.choice(len(X_train_scaled), min(sample_size, len(X_train_scaled)), replace=False)
X_sample = X_train_scaled[sample_indices]

print(f"Performing hierarchical clustering on sample of {len(X_sample)} merchants")
print(f"Testing linkage methods: {linkage_methods}\n")

# Store linkage results
linkage_results = {}

for method in linkage_methods:
    print(f"Computing {method} linkage...")
    linkage_matrix = linkage(X_sample, method=method)
    linkage_results[method] = linkage_matrix

print("\nLinkage computation complete!")

In [ ]:
# Visualize dendrograms for different linkage methods
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for ax, method in zip(axes, linkage_methods):
    dendrogram(linkage_results[method], ax=ax, no_labels=True, color_threshold=0)
    ax.set_title(f'Dendrogram - {method.capitalize()} Linkage', fontsize=14, fontweight='bold')
    ax.set_xlabel('Sample Index', fontsize=12)
    ax.set_ylabel('Distance', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDendrograms show the hierarchical structure of clusters.")
print("The height of each merge indicates the distance between clusters.")

In [ ]:
# Apply hierarchical clustering to full training data with optimal number of clusters
# Using Ward linkage (generally performs well and minimizes variance)

print("=== HIERARCHICAL CLUSTERING (WARD LINKAGE) ===\n")

# Compute linkage on full training data
linkage_train = linkage(X_train_scaled, method='ward')

# Cut the dendrogram to get clusters (using optimal_k from K-means)
n_clusters_hier = optimal_k
hierarchical_labels_train = fcluster(linkage_train, n_clusters_hier, criterion='maxclust')

# Assign labels to dataframes
df_train['cluster_hierarchical'] = hierarchical_labels_train - 1  # Convert to 0-indexed

# Predict on validation and test sets (using nearest cluster center approach)
# For hierarchical clustering on new data, we compute linkage with the training data
from scipy.spatial.distance import cdist

# Compute cluster centers from training data
hier_cluster_centers = []
for i in range(n_clusters_hier):
    cluster_mask = df_train['cluster_hierarchical'] == i
    center = X_train_scaled[cluster_mask].mean(axis=0)
    hier_cluster_centers.append(center)
hier_cluster_centers = np.array(hier_cluster_centers)

# Assign validation and test data to nearest cluster center
df_validate['cluster_hierarchical'] = cdist(X_validate_scaled, hier_cluster_centers).argmin(axis=1)
df_test['cluster_hierarchical'] = cdist(X_test_scaled, hier_cluster_centers).argmin(axis=1)

# Evaluate
train_silhouette_hier = silhouette_score(X_train_scaled, df_train['cluster_hierarchical'])
train_davies_bouldin_hier = davies_bouldin_score(X_train_scaled, df_train['cluster_hierarchical'])
validate_silhouette_hier = silhouette_score(X_validate_scaled, df_validate['cluster_hierarchical'])
validate_davies_bouldin_hier = davies_bouldin_score(X_validate_scaled, df_validate['cluster_hierarchical'])
test_silhouette_hier = silhouette_score(X_test_scaled, df_test['cluster_hierarchical'])
test_davies_bouldin_hier = davies_bouldin_score(X_test_scaled, df_test['cluster_hierarchical'])

print(f"Hierarchical Clustering with {n_clusters_hier} clusters (Ward Linkage)")
print(f"\nTraining Set Metrics:")
print(f"  Silhouette Score: {train_silhouette_hier:.4f}")
print(f"  Davies-Bouldin Index: {train_davies_bouldin_hier:.4f}")

print(f"\nValidation Set Metrics:")
print(f"  Silhouette Score: {validate_silhouette_hier:.4f}")
print(f"  Davies-Bouldin Index: {validate_davies_bouldin_hier:.4f}")

print(f"\nTest Set Metrics:")
print(f"  Silhouette Score: {test_silhouette_hier:.4f}")
print(f"  Davies-Bouldin Index: {test_davies_bouldin_hier:.4f}")

print(f"\nCluster Distribution (Train):")
print(df_train['cluster_hierarchical'].value_counts().sort_index())

In [ ]:
# Visualize hierarchical clustering results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets_hier = [
    (X_train_scaled, df_train['cluster_hierarchical'], 'Train'),
    (X_validate_scaled, df_validate['cluster_hierarchical'], 'Validate'),
    (X_test_scaled, df_test['cluster_hierarchical'], 'Test')
]

for ax, (X, clusters, title) in zip(axes, datasets_hier):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
    ax.scatter(hier_cluster_centers[:, 0], hier_cluster_centers[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centers')
    ax.set_xlabel(f'Feature 1: {feature_cols[0]}')
    ax.set_ylabel(f'Feature 2: {feature_cols[1]}')
    ax.set_title(f'{title} Set - Hierarchical Clustering ({n_clusters_hier} clusters)')
    ax.legend()
    plt.colorbar(scatter, ax=ax, label='Cluster')

plt.tight_layout()
plt.show()

### 7.2 Hierarchical Clustering on Z-Scored Data


In [ ]:
# Hierarchical clustering on z-scored data
print("=== HIERARCHICAL CLUSTERING - Z-SCORED DATA (WARD LINKAGE) ===\n")

# Compute linkage on z-scored training data
linkage_train_z = linkage(X_train_z, method='ward')

# Cut the dendrogram
n_clusters_hier_z = optimal_k_z
hierarchical_labels_train_z = fcluster(linkage_train_z, n_clusters_hier_z, criterion='maxclust')

df_train_z['cluster_hierarchical'] = hierarchical_labels_train_z - 1

# Compute cluster centers
hier_cluster_centers_z = []
for i in range(n_clusters_hier_z):
    cluster_mask = df_train_z['cluster_hierarchical'] == i
    center = X_train_z[cluster_mask].mean(axis=0)
    hier_cluster_centers_z.append(center)
hier_cluster_centers_z = np.array(hier_cluster_centers_z)

# Assign validation and test data
df_validate_z['cluster_hierarchical'] = cdist(X_validate_z, hier_cluster_centers_z).argmin(axis=1)
df_test_z['cluster_hierarchical'] = cdist(X_test_z, hier_cluster_centers_z).argmin(axis=1)

# Evaluate
train_silhouette_hier_z = silhouette_score(X_train_z, df_train_z['cluster_hierarchical'])
train_davies_bouldin_hier_z = davies_bouldin_score(X_train_z, df_train_z['cluster_hierarchical'])
validate_silhouette_hier_z = silhouette_score(X_validate_z, df_validate_z['cluster_hierarchical'])
validate_davies_bouldin_hier_z = davies_bouldin_score(X_validate_z, df_validate_z['cluster_hierarchical'])
test_silhouette_hier_z = silhouette_score(X_test_z, df_test_z['cluster_hierarchical'])
test_davies_bouldin_hier_z = davies_bouldin_score(X_test_z, df_test_z['cluster_hierarchical'])

print(f"Hierarchical Clustering with {n_clusters_hier_z} clusters (Ward Linkage, Z-Scored)")
print(f"\nTraining Set Metrics:")
print(f"  Silhouette Score: {train_silhouette_hier_z:.4f}")
print(f"  Davies-Bouldin Index: {train_davies_bouldin_hier_z:.4f}")

print(f"\nValidation Set Metrics:")
print(f"  Silhouette Score: {validate_silhouette_hier_z:.4f}")
print(f"  Davies-Bouldin Index: {validate_davies_bouldin_hier_z:.4f}")

print(f"\nTest Set Metrics:")
print(f"  Silhouette Score: {test_silhouette_hier_z:.4f}")
print(f"  Davies-Bouldin Index: {test_davies_bouldin_hier_z:.4f}")

print(f"\nCluster Distribution (Train):")
print(df_train_z['cluster_hierarchical'].value_counts().sort_index())

In [ ]:
# Visualize z-scored hierarchical clustering results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets_hier_z = [
    (X_train_z, df_train_z['cluster_hierarchical'], 'Train'),
    (X_validate_z, df_validate_z['cluster_hierarchical'], 'Validate'),
    (X_test_z, df_test_z['cluster_hierarchical'], 'Test')
]

for ax, (X, clusters, title) in zip(axes, datasets_hier_z):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
    ax.scatter(hier_cluster_centers_z[:, 0], hier_cluster_centers_z[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centers')
    ax.set_xlabel(f'Feature 1: {feature_cols_z[0]}')
    ax.set_ylabel(f'Feature 2: {feature_cols_z[1]}')
    ax.set_title(f'{title} Set - Hierarchical Clustering ({n_clusters_hier_z} clusters, Z-Scored)')
    ax.legend()
    plt.colorbar(scatter, ax=ax, label='Cluster')

plt.tight_layout()
plt.show()

---
## 9. Density-Based Clustering (DBSCAN)

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) identifies clusters based on density and can detect outliers.

**Key Advantages:**
- Finds arbitrarily shaped clusters
- Automatically identifies noise/outliers
- Doesn't require pre-specifying number of clusters

**Key Parameters:**
- **eps**: Maximum distance between neighbors
- **min_samples**: Minimum points to form a cluster

### 9.1 DBSCAN on Raw Data

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Determine optimal eps using k-distance graph
print("=== FINDING OPTIMAL EPS FOR DBSCAN ===\n")

# Use k=4 (2*dimensions) for k-nearest neighbors
k = 4
nbrs = NearestNeighbors(n_neighbors=k).fit(X_train_scaled)
distances, indices = nbrs.kneighbors(X_train_scaled)

# Sort distances
distances = np.sort(distances[:, k-1], axis=0)

# Plot k-distance graph
plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.xlabel('Data Points sorted by distance')
plt.ylabel(f'{k}-NN Distance')
plt.title('K-Distance Graph for Epsilon Selection')
plt.grid(True, alpha=0.3)
plt.axhline(y=np.percentile(distances, 95), color='r', linestyle='--', label='95th percentile')
plt.legend()
plt.show()

# Suggest eps based on elbow point (using 95th percentile as heuristic)
suggested_eps = np.percentile(distances, 95)
print(f"Suggested eps (95th percentile): {suggested_eps:.4f}")
print("Look for the 'elbow' in the k-distance graph to choose eps")

### 8.1.1 DBSCAN Hyperparameter Tuning (Raw Data)

DBSCAN has two key hyperparameters:
- **eps**: Maximum distance between two points to be considered neighbors
- **min_samples**: Minimum number of points to form a dense region (cluster)

We'll test combinations to find optimal values based on:
- **Silhouette Score**: Measures cluster quality (higher is better)
- **Number of Clusters**: Should be meaningful (not 1, not too many)
- **Noise Percentage**: Should be reasonable (<30%)

In [ ]:
# DBSCAN Hyperparameter Tuning - Raw Data
print("="*80)
print("DBSCAN HYPERPARAMETER TUNING - RAW DATA")
print("="*80)

# Define parameter grid
eps_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
min_samples_values = [3, 5, 7, 10]

tuning_results = []

print("\nTesting combinations of eps and min_samples...")
print("(This may take a moment)\n")

for eps in eps_values:
    for min_samp in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samp)
        labels = dbscan.fit_predict(X_train_scaled)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        noise_pct = n_noise / len(labels) * 100
        
        # Calculate silhouette score if possible
        if n_clusters > 1 and n_noise < len(labels) - 1:
            mask = labels != -1
            if mask.sum() > 1 and len(np.unique(labels[mask])) > 1:
                sil_score = silhouette_score(X_train_scaled[mask], labels[mask])
            else:
                sil_score = -1
        else:
            sil_score = -1
        
        tuning_results.append({
            'eps': eps,
            'min_samples': min_samp,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'noise_pct': noise_pct,
            'silhouette': sil_score
        })

# Convert to DataFrame
tuning_df = pd.DataFrame(tuning_results)

# Find best parameters based on different criteria
print("="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Filter valid results (at least 2 clusters, reasonable noise)
valid_results = tuning_df[(tuning_df['n_clusters'] >= 2) & 
                          (tuning_df['noise_pct'] < 30) & 
                          (tuning_df['silhouette'] > 0)]

if len(valid_results) > 0:
    # Best by silhouette score
    best_sil = valid_results.loc[valid_results['silhouette'].idxmax()]
    print(f"\nBest by Silhouette Score:")
    print(f"  eps={best_sil['eps']:.2f}, min_samples={int(best_sil['min_samples'])}")
    print(f"  Clusters: {int(best_sil['n_clusters'])}, Noise: {best_sil['noise_pct']:.1f}%, Silhouette: {best_sil['silhouette']:.4f}")
    
    # Best by balanced noise and silhouette
    valid_results['score'] = valid_results['silhouette'] * (1 - valid_results['noise_pct']/100)
    best_balanced = valid_results.loc[valid_results['score'].idxmax()]
    print(f"\nBest Balanced (Silhouette × Low Noise):")
    print(f"  eps={best_balanced['eps']:.2f}, min_samples={int(best_balanced['min_samples'])}")
    print(f"  Clusters: {int(best_balanced['n_clusters'])}, Noise: {best_balanced['noise_pct']:.1f}%, Silhouette: {best_balanced['silhouette']:.4f}")
else:
    print("\n⚠️  No valid parameter combinations found!")
    print("Try expanding the search range.")

# Show top 10 results
print(f"\n{'='*80}")
print("TOP 10 PARAMETER COMBINATIONS (by Silhouette Score):")
print(f"{'='*80}")
top_results = tuning_df[tuning_df['silhouette'] > 0].nlargest(10, 'silhouette')
print(top_results.to_string(index=False))

# Create heatmap of results
print(f"\n{'='*80}")
print("SILHOUETTE SCORE HEATMAP:")
print("(Higher is better; -1 indicates invalid configuration)")
print(f"{'='*80}\n")

# Pivot table for heatmap
pivot_sil = tuning_df.pivot_table(values='silhouette', index='min_samples', columns='eps')
print(pivot_sil.to_string())

print(f"\n{'='*80}")
print("NUMBER OF CLUSTERS HEATMAP:")
print(f"{'='*80}\n")
pivot_clusters = tuning_df.pivot_table(values='n_clusters', index='min_samples', columns='eps')
print(pivot_clusters.to_string())

In [ ]:
# Visualize DBSCAN hyperparameter tuning results - Raw Data
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Silhouette Score Heatmap
pivot_sil = tuning_df.pivot_table(values='silhouette', index='min_samples', columns='eps')
im1 = axes[0].imshow(pivot_sil, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(pivot_sil.columns)))
axes[0].set_xticklabels([f'{x:.2f}' for x in pivot_sil.columns], rotation=45)
axes[0].set_yticks(range(len(pivot_sil.index)))
axes[0].set_yticklabels(pivot_sil.index)
axes[0].set_xlabel('eps')
axes[0].set_ylabel('min_samples')
axes[0].set_title('Silhouette Score (Higher = Better)')
plt.colorbar(im1, ax=axes[0])

# Add text annotations
for i in range(len(pivot_sil.index)):
    for j in range(len(pivot_sil.columns)):
        val = pivot_sil.iloc[i, j]
        text_color = 'white' if val < 0 else 'black'
        axes[0].text(j, i, f'{val:.2f}', ha='center', va='center', color=text_color, fontsize=8)

# Number of Clusters Heatmap
pivot_clusters = tuning_df.pivot_table(values='n_clusters', index='min_samples', columns='eps')
im2 = axes[1].imshow(pivot_clusters, cmap='viridis', aspect='auto')
axes[1].set_xticks(range(len(pivot_clusters.columns)))
axes[1].set_xticklabels([f'{x:.2f}' for x in pivot_clusters.columns], rotation=45)
axes[1].set_yticks(range(len(pivot_clusters.index)))
axes[1].set_yticklabels(pivot_clusters.index)
axes[1].set_xlabel('eps')
axes[1].set_ylabel('min_samples')
axes[1].set_title('Number of Clusters')
plt.colorbar(im2, ax=axes[1])

# Add text annotations
for i in range(len(pivot_clusters.index)):
    for j in range(len(pivot_clusters.columns)):
        val = pivot_clusters.iloc[i, j]
        axes[1].text(j, i, f'{int(val)}', ha='center', va='center', color='white', fontsize=8)

# Noise Percentage Heatmap
pivot_noise = tuning_df.pivot_table(values='noise_pct', index='min_samples', columns='eps')
im3 = axes[2].imshow(pivot_noise, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=100)
axes[2].set_xticks(range(len(pivot_noise.columns)))
axes[2].set_xticklabels([f'{x:.2f}' for x in pivot_noise.columns], rotation=45)
axes[2].set_yticks(range(len(pivot_noise.index)))
axes[2].set_yticklabels(pivot_noise.index)
axes[2].set_xlabel('eps')
axes[2].set_ylabel('min_samples')
axes[2].set_title('Noise % (Lower = Better)')
plt.colorbar(im3, ax=axes[2])

# Add text annotations
for i in range(len(pivot_noise.index)):
    for j in range(len(pivot_noise.columns)):
        val = pivot_noise.iloc[i, j]
        text_color = 'white' if val > 50 else 'black'
        axes[2].text(j, i, f'{val:.0f}%', ha='center', va='center', color=text_color, fontsize=8)

plt.suptitle('DBSCAN Hyperparameter Tuning - Raw Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 8.1.2 Apply DBSCAN with Best Parameters (Raw Data)

Based on the tuning results above, update the `chosen_eps` and `chosen_min_samples` values in the next cell.

**Selection Guidelines:**
- ✅ Choose parameters with **high Silhouette Score** (>0.3)
- ✅ Ensure **multiple clusters** are found (not just 1)
- ✅ Keep **noise percentage reasonable** (<20-30%)
- ⚠️ Balance between cluster quality and noise tolerance

In [ ]:
# Auto-select best DBSCAN parameters from tuning results (Raw Data)
if len(valid_results) > 0:
    # Use the balanced best (silhouette × low noise)
    auto_eps = best_balanced['eps']
    auto_min_samples = int(best_balanced['min_samples'])
    
    print("="*80)
    print("RECOMMENDED PARAMETERS (Auto-Selected):")
    print("="*80)
    print(f"\nchosen_eps = {auto_eps}")
    print(f"chosen_min_samples = {auto_min_samples}")
    print(f"\nExpected Results:")
    print(f"  - Clusters: {int(best_balanced['n_clusters'])}")
    print(f"  - Noise: {best_balanced['noise_pct']:.1f}%")
    print(f"  - Silhouette: {best_balanced['silhouette']:.4f}")
    print("\nCopy these values to the next cell or adjust as needed.")
    print("="*80)
else:
    print("⚠️ No valid parameters found in tuning. Check the heatmaps and select manually.")

In [ ]:
# Apply DBSCAN with tuned parameters - Raw Data
# IMPORTANT: Update these values based on the hyperparameter tuning results above
# Look for the "Best by Silhouette Score" or "Best Balanced" recommendation

chosen_eps = 0.1  # Update based on tuning results
chosen_min_samples = 5  # Update based on tuning results

print(f"\n=== DBSCAN CLUSTERING - RAW DATA (eps={chosen_eps}, min_samples={chosen_min_samples}) ===\n")

# Fit DBSCAN on training data
dbscan = DBSCAN(eps=chosen_eps, min_samples=chosen_min_samples)
dbscan_labels_train = dbscan.fit_predict(X_train_scaled)

df_train['cluster_dbscan'] = dbscan_labels_train

# For validation and test sets, assign to nearest cluster (excluding noise)
# Get cluster centers (excluding noise points labeled as -1)
dbscan_cluster_centers = []
unique_labels = set(dbscan_labels_train)
if -1 in unique_labels:
    unique_labels.remove(-1)

for label in sorted(unique_labels):
    cluster_mask = dbscan_labels_train == label
    center = X_train_scaled[cluster_mask].mean(axis=0)
    dbscan_cluster_centers.append(center)

if len(dbscan_cluster_centers) > 0:
    dbscan_cluster_centers = np.array(dbscan_cluster_centers)
    
    # Assign validation and test data to nearest cluster
    df_validate['cluster_dbscan'] = cdist(X_validate_scaled, dbscan_cluster_centers).argmin(axis=1)
    df_test['cluster_dbscan'] = cdist(X_test_scaled, dbscan_cluster_centers).argmin(axis=1)
    
    # Evaluate (excluding noise points from training)
    train_mask = dbscan_labels_train != -1
    # Check for at least 2 non-noise points AND at least 2 unique clusters
    if train_mask.sum() > 1 and len(np.unique(dbscan_labels_train[train_mask])) > 1:
        train_silhouette_dbscan = silhouette_score(X_train_scaled[train_mask], dbscan_labels_train[train_mask])
        train_davies_bouldin_dbscan = davies_bouldin_score(X_train_scaled[train_mask], dbscan_labels_train[train_mask])
    else:
        train_silhouette_dbscan = -1
        train_davies_bouldin_dbscan = -1
    
    # Check if validate/test have at least 2 unique clusters (happens when training found only 1 cluster)
    if len(np.unique(df_validate['cluster_dbscan'])) > 1:
        validate_silhouette_dbscan = silhouette_score(X_validate_scaled, df_validate['cluster_dbscan'])
        validate_davies_bouldin_dbscan = davies_bouldin_score(X_validate_scaled, df_validate['cluster_dbscan'])
    else:
        validate_silhouette_dbscan = -1
        validate_davies_bouldin_dbscan = -1
    
    if len(np.unique(df_test['cluster_dbscan'])) > 1:
        test_silhouette_dbscan = silhouette_score(X_test_scaled, df_test['cluster_dbscan'])
        test_davies_bouldin_dbscan = davies_bouldin_score(X_test_scaled, df_test['cluster_dbscan'])
    else:
        test_silhouette_dbscan = -1
        test_davies_bouldin_dbscan = -1
    
    n_clusters_found = len(unique_labels)
    n_noise = list(dbscan_labels_train).count(-1)
    
    print(f"DBSCAN Results:")
    print(f"  Clusters found: {n_clusters_found}")
    print(f"  Noise points: {n_noise} ({n_noise/len(dbscan_labels_train)*100:.1f}%)")
    
    print(f"\nTraining Set Metrics (excluding noise):")
    print(f"  Silhouette Score: {f'{train_silhouette_dbscan:.4f}' if train_silhouette_dbscan > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{train_davies_bouldin_dbscan:.4f}' if train_davies_bouldin_dbscan > 0 else 'N/A'}")
    
    print(f"\nValidation Set Metrics:")
    print(f"  Silhouette Score: {f'{validate_silhouette_dbscan:.4f}' if validate_silhouette_dbscan > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{validate_davies_bouldin_dbscan:.4f}' if validate_davies_bouldin_dbscan > 0 else 'N/A'}")
    
    print(f"\nTest Set Metrics:")
    print(f"  Silhouette Score: {f'{test_silhouette_dbscan:.4f}' if test_silhouette_dbscan > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{test_davies_bouldin_dbscan:.4f}' if test_davies_bouldin_dbscan > 0 else 'N/A'}")
    
    print(f"\nCluster Distribution (Train, including noise as -1):")
    print(df_train['cluster_dbscan'].value_counts().sort_index())
else:
    print("No clusters found - all points classified as noise!")
    print("⚠️ Try increasing eps or decreasing min_samples")

In [ ]:
# Visualize DBSCAN results
if len(dbscan_cluster_centers) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    datasets_dbscan = [
        (X_train_scaled, df_train['cluster_dbscan'], 'Train'),
        (X_validate_scaled, df_validate['cluster_dbscan'], 'Validate'),
        (X_test_scaled, df_test['cluster_dbscan'], 'Test')
    ]
    
    for ax, (X, clusters, title) in zip(axes, datasets_dbscan):
        # Use different color for noise points (-1)
        scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
        if len(dbscan_cluster_centers) > 0:
            ax.scatter(dbscan_cluster_centers[:, 0], dbscan_cluster_centers[:, 1], 
                       c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centers')
        ax.set_xlabel(f'Feature 1: {feature_cols[0]}')
        ax.set_ylabel(f'Feature 2: {feature_cols[1]}')
        ax.set_title(f'{title} Set - DBSCAN (eps={chosen_eps}, noise=-1)')
        ax.legend()
        plt.colorbar(scatter, ax=ax, label='Cluster')
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot visualize - no clusters found")

---
## 9.2 DBSCAN on Z-Scored Data

### 9.2.1 DBSCAN Hyperparameter Tuning (Z-Scored Data)

For z-scored data (normalized with mean=0, std=1), we need **smaller eps values** than raw data because distances are compressed. We'll test appropriate ranges for this normalized space.

In [ ]:
# Find optimal eps for z-scored data
print("=== FINDING OPTIMAL EPS FOR DBSCAN (Z-SCORED DATA) ===\n")

k = 4
nbrs_z = NearestNeighbors(n_neighbors=k).fit(X_train_z)
distances_z, indices_z = nbrs_z.kneighbors(X_train_z)
distances_z = np.sort(distances_z[:, k-1], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(distances_z)
plt.xlabel('Data Points sorted by distance')
plt.ylabel(f'{k}-NN Distance')
plt.title('K-Distance Graph for Epsilon Selection (Z-Scored Data)')
plt.grid(True, alpha=0.3)
plt.axhline(y=np.percentile(distances_z, 95), color='r', linestyle='--', label='95th percentile')
plt.legend()
plt.show()

suggested_eps_z = np.percentile(distances_z, 95)
print(f"Suggested eps (95th percentile): {suggested_eps_z:.4f}")

In [ ]:
# DBSCAN Hyperparameter Tuning - Z-Scored Data
print("="*80)
print("DBSCAN HYPERPARAMETER TUNING - Z-SCORED DATA")
print("="*80)

# For z-scored data, use SMALLER eps values (data is normalized)
eps_values_z = [0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 0.7, 1.0]
min_samples_values_z = [3, 5, 7, 10]

tuning_results_z = []

print("\nTesting combinations of eps and min_samples...")
print("(This may take a moment)\n")

for eps in eps_values_z:
    for min_samp in min_samples_values_z:
        dbscan_z = DBSCAN(eps=eps, min_samples=min_samp)
        labels_z = dbscan_z.fit_predict(X_train_z)
        
        n_clusters = len(set(labels_z)) - (1 if -1 in labels_z else 0)
        n_noise = list(labels_z).count(-1)
        noise_pct = n_noise / len(labels_z) * 100
        
        # Calculate silhouette score if possible
        if n_clusters > 1 and n_noise < len(labels_z) - 1:
            mask = labels_z != -1
            if mask.sum() > 1 and len(np.unique(labels_z[mask])) > 1:
                sil_score = silhouette_score(X_train_z[mask], labels_z[mask])
            else:
                sil_score = -1
        else:
            sil_score = -1
        
        tuning_results_z.append({
            'eps': eps,
            'min_samples': min_samp,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'noise_pct': noise_pct,
            'silhouette': sil_score
        })

# Convert to DataFrame
tuning_df_z = pd.DataFrame(tuning_results_z)

# Find best parameters based on different criteria
print("="*80)
print("TUNING RESULTS SUMMARY")
print("="*80)

# Filter valid results (at least 2 clusters, reasonable noise)
valid_results_z = tuning_df_z[(tuning_df_z['n_clusters'] >= 2) & 
                              (tuning_df_z['noise_pct'] < 30) & 
                              (tuning_df_z['silhouette'] > 0)]

if len(valid_results_z) > 0:
    # Best by silhouette score
    best_sil_z = valid_results_z.loc[valid_results_z['silhouette'].idxmax()]
    print(f"\nBest by Silhouette Score:")
    print(f"  eps={best_sil_z['eps']:.2f}, min_samples={int(best_sil_z['min_samples'])}")
    print(f"  Clusters: {int(best_sil_z['n_clusters'])}, Noise: {best_sil_z['noise_pct']:.1f}%, Silhouette: {best_sil_z['silhouette']:.4f}")
    
    # Best by balanced noise and silhouette
    valid_results_z['score'] = valid_results_z['silhouette'] * (1 - valid_results_z['noise_pct']/100)
    best_balanced_z = valid_results_z.loc[valid_results_z['score'].idxmax()]
    print(f"\nBest Balanced (Silhouette × Low Noise):")
    print(f"  eps={best_balanced_z['eps']:.2f}, min_samples={int(best_balanced_z['min_samples'])}")
    print(f"  Clusters: {int(best_balanced_z['n_clusters'])}, Noise: {best_balanced_z['noise_pct']:.1f}%, Silhouette: {best_balanced_z['silhouette']:.4f}")
else:
    print("\n⚠️  No valid parameter combinations found!")
    print("Try expanding the search range.")

# Show top 10 results
print(f"\n{'='*80}")
print("TOP 10 PARAMETER COMBINATIONS (by Silhouette Score):")
print(f"{'='*80}")
top_results_z = tuning_df_z[tuning_df_z['silhouette'] > 0].nlargest(10, 'silhouette')
print(top_results_z.to_string(index=False))

# Create heatmap of results
print(f"\n{'='*80}")
print("SILHOUETTE SCORE HEATMAP:")
print("(Higher is better; -1 indicates invalid configuration)")
print(f"{'='*80}\n")

# Pivot table for heatmap
pivot_sil_z = tuning_df_z.pivot_table(values='silhouette', index='min_samples', columns='eps')
print(pivot_sil_z.to_string())

print(f"\n{'='*80}")
print("NUMBER OF CLUSTERS HEATMAP:")
print(f"{'='*80}\n")
pivot_clusters_z = tuning_df_z.pivot_table(values='n_clusters', index='min_samples', columns='eps')
print(pivot_clusters_z.to_string())

In [ ]:
# Visualize DBSCAN hyperparameter tuning results - Z-Scored Data
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Silhouette Score Heatmap
pivot_sil_z = tuning_df_z.pivot_table(values='silhouette', index='min_samples', columns='eps')
im1 = axes[0].imshow(pivot_sil_z, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(pivot_sil_z.columns)))
axes[0].set_xticklabels([f'{x:.2f}' for x in pivot_sil_z.columns], rotation=45)
axes[0].set_yticks(range(len(pivot_sil_z.index)))
axes[0].set_yticklabels(pivot_sil_z.index)
axes[0].set_xlabel('eps')
axes[0].set_ylabel('min_samples')
axes[0].set_title('Silhouette Score (Higher = Better)')
plt.colorbar(im1, ax=axes[0])

# Add text annotations
for i in range(len(pivot_sil_z.index)):
    for j in range(len(pivot_sil_z.columns)):
        val = pivot_sil_z.iloc[i, j]
        text_color = 'white' if val < 0 else 'black'
        axes[0].text(j, i, f'{val:.2f}', ha='center', va='center', color=text_color, fontsize=8)

# Number of Clusters Heatmap
pivot_clusters_z = tuning_df_z.pivot_table(values='n_clusters', index='min_samples', columns='eps')
im2 = axes[1].imshow(pivot_clusters_z, cmap='viridis', aspect='auto')
axes[1].set_xticks(range(len(pivot_clusters_z.columns)))
axes[1].set_xticklabels([f'{x:.2f}' for x in pivot_clusters_z.columns], rotation=45)
axes[1].set_yticks(range(len(pivot_clusters_z.index)))
axes[1].set_yticklabels(pivot_clusters_z.index)
axes[1].set_xlabel('eps')
axes[1].set_ylabel('min_samples')
axes[1].set_title('Number of Clusters')
plt.colorbar(im2, ax=axes[1])

# Add text annotations
for i in range(len(pivot_clusters_z.index)):
    for j in range(len(pivot_clusters_z.columns)):
        val = pivot_clusters_z.iloc[i, j]
        axes[1].text(j, i, f'{int(val)}', ha='center', va='center', color='white', fontsize=8)

# Noise Percentage Heatmap
pivot_noise_z = tuning_df_z.pivot_table(values='noise_pct', index='min_samples', columns='eps')
im3 = axes[2].imshow(pivot_noise_z, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=100)
axes[2].set_xticks(range(len(pivot_noise_z.columns)))
axes[2].set_xticklabels([f'{x:.2f}' for x in pivot_noise_z.columns], rotation=45)
axes[2].set_yticks(range(len(pivot_noise_z.index)))
axes[2].set_yticklabels(pivot_noise_z.index)
axes[2].set_xlabel('eps')
axes[2].set_ylabel('min_samples')
axes[2].set_title('Noise % (Lower = Better)')
plt.colorbar(im3, ax=axes[2])

# Add text annotations
for i in range(len(pivot_noise_z.index)):
    for j in range(len(pivot_noise_z.columns)):
        val = pivot_noise_z.iloc[i, j]
        text_color = 'white' if val > 50 else 'black'
        axes[2].text(j, i, f'{val:.0f}%', ha='center', va='center', color=text_color, fontsize=8)

plt.suptitle('DBSCAN Hyperparameter Tuning - Z-Scored Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 8.2.2 Apply DBSCAN with Best Parameters (Z-Scored Data)

Based on the tuning results above, update the `chosen_eps_z` and `chosen_min_samples_z` values in the next cell.

**Remember:** Z-scored data requires **smaller eps values** (typically 0.05-0.3) than raw data!

In [ ]:
# Auto-select best DBSCAN parameters from tuning results (Z-Scored Data)
if len(valid_results_z) > 0:
    # Use the balanced best (silhouette × low noise)
    auto_eps_z = best_balanced_z['eps']
    auto_min_samples_z = int(best_balanced_z['min_samples'])
    
    print("="*80)
    print("RECOMMENDED PARAMETERS (Auto-Selected):")
    print("="*80)
    print(f"\nchosen_eps_z = {auto_eps_z}")
    print(f"chosen_min_samples_z = {auto_min_samples_z}")
    print(f"\nExpected Results:")
    print(f"  - Clusters: {int(best_balanced_z['n_clusters'])}")
    print(f"  - Noise: {best_balanced_z['noise_pct']:.1f}%")
    print(f"  - Silhouette: {best_balanced_z['silhouette']:.4f}")
    print("\nCopy these values to the next cell or adjust as needed.")
    print("="*80)
else:
    print("⚠️ No valid parameters found in tuning. Check the heatmaps and select manually.")

In [ ]:
# Apply DBSCAN with tuned parameters - Z-Scored Data
# IMPORTANT: Update these values based on the hyperparameter tuning results above
# Look for the "Best by Silhouette Score" or "Best Balanced" recommendation
# NOTE: For z-scored data, use SMALLER eps values than raw data

chosen_eps_z = 0.5  # Update based on tuning results (typically 0.05-0.3 for z-scored)
chosen_min_samples_z = 3  # Update based on tuning results

print(f"\n=== DBSCAN CLUSTERING - Z-SCORED (eps={chosen_eps_z}, min_samples={chosen_min_samples_z}) ===\n")

dbscan_z = DBSCAN(eps=chosen_eps_z, min_samples=chosen_min_samples_z)
dbscan_labels_train_z = dbscan_z.fit_predict(X_train_z)

df_train_z['cluster_dbscan'] = dbscan_labels_train_z

# Get cluster centers (excluding noise)
dbscan_cluster_centers_z = []
unique_labels_z = set(dbscan_labels_train_z)
if -1 in unique_labels_z:
    unique_labels_z.remove(-1)

for label in sorted(unique_labels_z):
    cluster_mask = dbscan_labels_train_z == label
    center = X_train_z[cluster_mask].mean(axis=0)
    dbscan_cluster_centers_z.append(center)

if len(dbscan_cluster_centers_z) > 0:
    dbscan_cluster_centers_z = np.array(dbscan_cluster_centers_z)
    
    df_validate_z['cluster_dbscan'] = cdist(X_validate_z, dbscan_cluster_centers_z).argmin(axis=1)
    df_test_z['cluster_dbscan'] = cdist(X_test_z, dbscan_cluster_centers_z).argmin(axis=1)
    
    train_mask_z = dbscan_labels_train_z != -1
    # Check for at least 2 non-noise points AND at least 2 unique clusters
    if train_mask_z.sum() > 1 and len(np.unique(dbscan_labels_train_z[train_mask_z])) > 1:
        train_silhouette_dbscan_z = silhouette_score(X_train_z[train_mask_z], dbscan_labels_train_z[train_mask_z])
        train_davies_bouldin_dbscan_z = davies_bouldin_score(X_train_z[train_mask_z], dbscan_labels_train_z[train_mask_z])
    else:
        train_silhouette_dbscan_z = -1
        train_davies_bouldin_dbscan_z = -1
    
    # Check if validate/test have at least 2 unique clusters (happens when training found only 1 cluster)
    if len(np.unique(df_validate_z['cluster_dbscan'])) > 1:
        validate_silhouette_dbscan_z = silhouette_score(X_validate_z, df_validate_z['cluster_dbscan'])
        validate_davies_bouldin_dbscan_z = davies_bouldin_score(X_validate_z, df_validate_z['cluster_dbscan'])
    else:
        validate_silhouette_dbscan_z = -1
        validate_davies_bouldin_dbscan_z = -1
    
    if len(np.unique(df_test_z['cluster_dbscan'])) > 1:
        test_silhouette_dbscan_z = silhouette_score(X_test_z, df_test_z['cluster_dbscan'])
        test_davies_bouldin_dbscan_z = davies_bouldin_score(X_test_z, df_test_z['cluster_dbscan'])
    else:
        test_silhouette_dbscan_z = -1
        test_davies_bouldin_dbscan_z = -1
    
    n_clusters_found_z = len(unique_labels_z)
    n_noise_z = list(dbscan_labels_train_z).count(-1)
    
    print(f"DBSCAN Results (Z-Scored):")
    print(f"  Clusters found: {n_clusters_found_z}")
    print(f"  Noise points: {n_noise_z} ({n_noise_z/len(dbscan_labels_train_z)*100:.1f}%)")
    
    print(f"\nTraining Set Metrics (excluding noise):")
    print(f"  Silhouette Score: {f'{train_silhouette_dbscan_z:.4f}' if train_silhouette_dbscan_z > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{train_davies_bouldin_dbscan_z:.4f}' if train_davies_bouldin_dbscan_z > 0 else 'N/A'}")
    
    print(f"\nValidation Set Metrics:")
    print(f"  Silhouette Score: {f'{validate_silhouette_dbscan_z:.4f}' if validate_silhouette_dbscan_z > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{validate_davies_bouldin_dbscan_z:.4f}' if validate_davies_bouldin_dbscan_z > 0 else 'N/A'}")
    
    print(f"\nTest Set Metrics:")
    print(f"  Silhouette Score: {f'{test_silhouette_dbscan_z:.4f}' if test_silhouette_dbscan_z > 0 else 'N/A'}")
    print(f"  Davies-Bouldin Index: {f'{test_davies_bouldin_dbscan_z:.4f}' if test_davies_bouldin_dbscan_z > 0 else 'N/A'}")
    
    print(f"\nCluster Distribution (Train, including noise as -1):")
    print(df_train_z['cluster_dbscan'].value_counts().sort_index())
else:
    print("No clusters found - all points classified as noise!")
    print("⚠️ Try increasing eps or decreasing min_samples")

In [ ]:
# Diagnostic: Check DBSCAN z-scored results
print("\n=== DBSCAN Z-SCORED DIAGNOSTIC ===\n")
print(f"Number of clusters found: {n_clusters_found_z}")
print(f"Number of noise points: {n_noise_z}")
print(f"Unique cluster labels: {sorted(unique_labels_z)}")
print(f"\nNumber of unique clusters (non-noise): {len(np.unique(dbscan_labels_train_z[dbscan_labels_train_z != -1]))}")

print("\n--- RECOMMENDATION ---")
if n_clusters_found_z == 1:
    print("⚠️  DBSCAN found only 1 cluster with eps=0.5")
    print("This is because eps=0.5 is too LARGE for z-scored data.")
    print("In z-scored space, data is normalized (mean=0, std=1).")
    print("Try SMALLER epsilon values: 0.05, 0.1, 0.15, 0.2, 0.3")
    print("\nBased on the eps testing above, choose an eps value that gives:")
    print("  - Multiple clusters (not just 1)")
    print("  - Reasonable silhouette score")
    print("  - Acceptable noise percentage (<20%)")
else:
    print(f"✓ DBSCAN found {n_clusters_found_z} clusters")
    print("Continue with analysis!")

In [ ]:
# Visualize DBSCAN results (z-scored)
if len(dbscan_cluster_centers_z) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    datasets_dbscan_z = [
        (X_train_z, df_train_z['cluster_dbscan'], 'Train'),
        (X_validate_z, df_validate_z['cluster_dbscan'], 'Validate'),
        (X_test_z, df_test_z['cluster_dbscan'], 'Test')
    ]
    
    for ax, (X, clusters, title) in zip(axes, datasets_dbscan_z):
        scatter = ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
        if len(dbscan_cluster_centers_z) > 0:
            ax.scatter(dbscan_cluster_centers_z[:, 0], dbscan_cluster_centers_z[:, 1], 
                       c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centers')
        ax.set_xlabel(f'Feature 1: {feature_cols_z[0]}')
        ax.set_ylabel(f'Feature 2: {feature_cols_z[1]}')
        ax.set_title(f'{title} Set - DBSCAN Z-Scored (eps={chosen_eps_z}, noise=-1)')
        ax.legend()
        plt.colorbar(scatter, ax=ax, label='Cluster')
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot visualize - no clusters found")

---
## 10. Clustering Methods Comparison

Compare all clustering methods side-by-side.


In [ ]:
# Comprehensive comparison of all clustering methods
print("="*100)
print("COMPREHENSIVE CLUSTERING METHODS COMPARISON")
print("="*100)

comparison_data = {
    'Method': [
        'K-Means (Raw)',
        'K-Means (Z-Scored)',
        'Hierarchical (Raw)',
        'Hierarchical (Z-Scored)',
        'DBSCAN (Raw)',
        'DBSCAN (Z-Scored)'
    ],
    'Train_Silhouette': [
        train_silhouette,
        train_silhouette_z,
        train_silhouette_hier,
        train_silhouette_hier_z,
        train_silhouette_dbscan if 'train_silhouette_dbscan' in locals() else -1,
        train_silhouette_dbscan_z if 'train_silhouette_dbscan_z' in locals() else -1
    ],
    'Train_DaviesBouldin': [
        train_davies_bouldin,
        train_davies_bouldin_z,
        train_davies_bouldin_hier,
        train_davies_bouldin_hier_z,
        train_davies_bouldin_dbscan if 'train_davies_bouldin_dbscan' in locals() else -1,
        train_davies_bouldin_dbscan_z if 'train_davies_bouldin_dbscan_z' in locals() else -1
    ],
    'Validate_Silhouette': [
        validate_silhouette,
        validate_silhouette_z,
        validate_silhouette_hier,
        validate_silhouette_hier_z,
        validate_silhouette_dbscan if 'validate_silhouette_dbscan' in locals() else -1,
        validate_silhouette_dbscan_z if 'validate_silhouette_dbscan_z' in locals() else -1
    ],
    'Validate_DaviesBouldin': [
        validate_davies_bouldin,
        validate_davies_bouldin_z,
        validate_davies_bouldin_hier,
        validate_davies_bouldin_hier_z,
        validate_davies_bouldin_dbscan if 'validate_davies_bouldin_dbscan' in locals() else -1,
        validate_davies_bouldin_dbscan_z if 'validate_davies_bouldin_dbscan_z' in locals() else -1
    ],
    'Test_Silhouette': [
        test_silhouette,
        test_silhouette_z,
        test_silhouette_hier,
        test_silhouette_hier_z,
        test_silhouette_dbscan if 'test_silhouette_dbscan' in locals() else -1,
        test_silhouette_dbscan_z if 'test_silhouette_dbscan_z' in locals() else -1
    ],
    'Test_DaviesBouldin': [
        test_davies_bouldin,
        test_davies_bouldin_z,
        test_davies_bouldin_hier,
        test_davies_bouldin_hier_z,
        test_davies_bouldin_dbscan if 'test_davies_bouldin_dbscan' in locals() else -1,
        test_davies_bouldin_dbscan_z if 'test_davies_bouldin_dbscan_z' in locals() else -1
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

print("\n" + "="*100)
print("INTERPRETATION:")
print("  - Silhouette Score: Higher is better (range -1 to 1)")
print("  - Davies-Bouldin Index: Lower is better (range 0 to ∞)")
print("  - N/A or negative values indicate insufficient data for metric calculation")
print("="*100)